<a href="https://colab.research.google.com/github/yassine-fetoui/artificial-intelligence/blob/main/Variational_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle


In [ ]:
!mkdir ~/.kaggle
!cp '/content/drive/MyDrive/kaggle.json' ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!kaggle datasets download -d scribbless/another-anime-face-dataset

In [ ]:
!pip install keras

In [ ]:
!pip install tensorflow-gpu

!pip install python-opencv
!pip install matplotlib
!pip install numpy

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os


In [ ]:
image_dir ="/content/animefaces256cleaner"
images = [os.path.join(image_dir,image)for image in os.listdir(image_dir)]
images[:2]

In [ ]:
#preprocessing
image_size=64

def preprocessing(image):
  image=tf.io.read_file(image)
  image=tf.io.decode_jpeg(image)
  image=tf.cast(image,tf.float32)
  image=tf.image.resize(image,(image_size,image_size))
  image=image/255.0
  image=tf.reshape(image , shape=(image_size,image_size,3))
  return image

In [ ]:
batch_size=128
training_dataset = tf.data.Dataset.from_tensor_slices((images))
training_dataset = training_dataset.map(preprocessing)
training_dataset = training_dataset.shuffle(1000).batch(batch_size)

In [ ]:
len(training_dataset)

In [ ]:
# visualize some of them
fig, axes =plt.subplots(5,5,figsize=(14,14))
sample = training_dataset.unbatch().take(25)
sample= [image for image in sample]
idx= 0
for row in range(5):
  for column in range(5):
    axes[row,column ].imshow(sample[idx])
    idx+=1


In [ ]:
latent_dim = 512
from keras.models import Sequential , Model
from keras.layers import LeakyReLU

from keras.layers import Dense , Conv2D , Conv2DTranspose , Input , Flatten, BatchNormalization , Lambda, Reshape # Conv2DTranspose = upsamplong
from keras.activations import selu
from keras.layers import Multiply, Add
from keras.optimizers import Adam

from keras import backend as k


In [ ]:
k.clear_session()

In [ ]:
encoder_input = Input(shape=(64,64,3))
x = Conv2D(32 ,kernel_size=5, activation=LeakyReLU(0.02) ,strides=1,padding="same")(encoder_input)
x = BatchNormalization()(x)
filter_size=[64,128,256,512]
for i in filter_size :
  x = Conv2D(i ,kernel_size=5, activation=LeakyReLU(0.02) ,strides=2,padding="same")(encoder_input)
  x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(1024,activation=selu)(x)
encoder_output =  BatchNormalization()(x)
#sampling layer
mu= Dense(latent_dim)(encoder_output)
log_var=Dense(latent_dim)(encoder_output)

epsilon = k.random_normal(shape=(tf.shape(mu)[0],tf.shape(mu)[1]))
sigma = tf.exp(0.5 * log_var)

z_eps=Multiply()([epsilon, sigma])
z=Add()([mu,z_eps])
encoder=Model(encoder_input,outputs=[mu,log_var,z], name='encoder')
encoder.summary()

In [ ]:
#build the decoder
decoder = Sequential()
decoder.add(Dense(1024,activation=selu, input_shape=(latent_dim,)))
decoder.add(BatchNormalization())

decoder.add(Dense(8192,activation=selu))
decoder.add(Reshape((4,4,512)))

decoder.add(Conv2DTranspose(256,(5,5),activation= LeakyReLU(0.02),strides=2,padding='same'))
decoder.add(BatchNormalization())

decoder.add(Conv2DTranspose(128,(5,5),activation= LeakyReLU(0.02),strides=2,padding='same'))
decoder.add(BatchNormalization())

decoder.add(Conv2DTranspose(64,(5,5),activation= LeakyReLU(0.02),strides=2,padding='same'))
decoder.add(BatchNormalization())

decoder.add(Conv2DTranspose(32,(5,5),activation= LeakyReLU(0.02),strides=2,padding='same'))
decoder.add(BatchNormalization())

decoder.add(Conv2DTranspose(3,(5,5),activation="sigmoid",strides=1,padding='same'))
decoder.add(BatchNormalization())

decoder.summary()



In [ ]:
# make loss function
# vae loss = reconstruction loss + kl div

def reconstruction_loss(y ,y_pred):
  return tf.reduce_mean(tf.square(y -y_pred))

def kl_loss(mu,log_var):
  loss=-0.5* tf.reduce_mean(1 + log_var - tf.square(mu) - tf.exp(log_var)) # regulerize
  return loss

def vae_loss(y_true, y_pred , mu,  log_var ) :
  return reconstruction_loss(y_true  ,y_pred) + (1/64*64) * kl_loss(mu,log_var)

In [ ]:
mu , log_var, z= encoder(encoder_input)
reconstructed= decoder(z)
model = Model(encoder_input, reconstructed , name='VAE')
loss=kl_loss(mu,log_var)
model.add_loss(loss)
model.summary()


In [ ]:
def save_images(model,epoch , step , input_):
  prediction = model.predict(input_)
  fig,axes=plt.subplots(5,5,figsize=(14,14))
  idx = 0
  for row in range(5):
    for column in range(5) :
      image = prediction[idx] * 255
      image = image.astype('int32')
      axes[row, column].imshow(image)
      axes[row,column].axis("off") # without coordenation
      idx+=1
  output_path="/content/output"
  if not os.path.exists(output_path):
    os.mkdir(output_path)
    plt.savefig(output_path + "Epoch_ {:04d}_step_{:04d}.jpg".format(epoch , step))
    plt.close()

In [ ]:
#train
from keras.optimizers import Adam
random_vector = tf.random.normal(shape=(25,latent_dim,))
save_images(decoder,0,0,random_vector)
mse_losses= []
kl_losses = []

optimizer = Adam(0.00001,0.5)
epochs=100
for epoch in range(1,epochs+1):
  print("Epoch",epoch)
  for step , training_batch in enumerate(training_dataset) :
    with tf.GradientTape() as tape:
      reconstructed = model(training_batch)
      y_true = tf.reshape(training_batch,shape= [-1])
      y_pred = tf.reshape(reconstructed , shape = [-1])

      mse_loss = reconstruction_loss(y_true,y_pred)
      mse_losses.append(mse_loss.numpy())

      train_loss = 0.01* mse_loss

      grads = tape.gradient(train_loss, model.trainable_variables)
      optimizer.apply_gradients(zip(grads,model.trainable_variables))

      if step % 10 ==0  :
        save_images(decoder,epoch ,step,random_vector)
      print("Epoch %s -step %s- MSE loss: %s -KL loss: %s"%(epoch ,step,mse_loss.numpy(),kl.numpy()))


